In [1]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.models import *

from src.utilities import *
from src.bubble import *

import openai
import instructor

#openai.api_key = os.environ["OPENAI_API_KEY"]


In [2]:
context = {
    "company": "Darty",
    "context": None,
    }

In [3]:
try:
    COMPANY_ID = bubble_client.get_objects("Company",[
                BubbleField("Name") == context["company"],
            ])[0]['_id']
    print("Retrieved company", COMPANY_ID)
    
except:
    COMPANY_ID = bubble_client.create("Company",
        {
            "Name": context["company"],
        })
    print("Created company", COMPANY_ID)

Retrieved company 1707313014508x102198350946437700


In [4]:
with open('Prompts/prompt_company_informations.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_company_informations = template.invoke(context)

class CompanyContext(BaseModel):
    context: str = Field(description="Contexte de l'entreprise.")

    def __str__(self):
        return self.context

res = get_analysis(client, prompt_company_informations, CompanyContext)
context['context'] = res.context
print(res)

Darty est une entreprise française spécialisée dans la distribution d'électroménager, d'informatique et d'électronique grand public. Elle a été fondée en 1957 par Natan Darty et est aujourd'hui présente dans plusieurs pays européens. En 2016, Darty a été rachetée par la Fnac, créant ainsi un grand groupe français de distribution de produits culturels et techniques. Plus récemment, en 2021, le groupe Fnac Darty a annoncé son intention de se rapprocher de l'entreprise britannique Dixons Carphone, ce qui pourrait donner naissance à un leader paneuropéen de la distribution de produits technologiques. Parmi les concurrents de Darty, on peut citer Boulanger, But, et Conforama.


In [5]:
with open('Prompts/prompt_categories_creation_example.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_categories_creation = template.invoke(context)

class InsightSubCategory(BaseModel):
    name: str = Field(description="Nom de la sous-catégorie.")
    description: str = Field(description="Phrase de description de la sous-catégorie.")
    id: str
    def __str__(self):
        return '\n- ' + self.name + " : " + self.description

class InsightCategory(BaseModel):
    name: str = Field(description="Nom de la catégorie.")
    description: str = Field(description="Phrase de description de la catégorie.")
    sub_categories: List[InsightSubCategory] = Field(description="Sous-catégories.")
    id: str

    def __str__(self):
        return self.name + " : " + self.description + ''.join([str(x) for x in self.sub_categories])

class InsightCategories(BaseModel):
    categories: List[InsightCategory] = Field(description="Catégories parmi lesquels les insights pourront être classifiées.")

    def __str__(self):
        return '\n\n'.join([str(x) for x in self.categories])

res = get_analysis(client, prompt_categories_creation, InsightCategories)
print(res)

In [ ]:
for categorie in res.categories:
    print(categorie)

Accueil en magasin : Les clients partagent leur opinion sur l'accueil réservé par les vendeurs en magasin.
- Accueil chaleureux : Le client est accueilli chaleureusement.
- Aucun accueil : Le client est ignoré.
Navigation sur le site web : Les clients expriment leur ressenti sur la facilité de navigation du site web.
- Facile de navigation : Le site web est facile à utiliser.
- Difficile de navigation : Le site web est compliqué à utiliser.
Qualité des produits : Les clients commentent la qualité des produits proposés en magasin.
- Produits de qualité : Les produits sont de bonne qualité.
- Produits de mauvaise qualité : Les produits sont de mauvaise qualité.
Temps d'attente avant d'être reçu par un vendeur : Les clients évaluent le temps d'attente avant d'être reçus par un vendeur.
- Attente raisonnable : Le temps d'attente est raisonnable.
- Attente trop longue : Le temps d'attente est trop long.
Processus d'achat en magasin : Les clients donnent leur avis sur le processus d'achat en

In [ ]:
for categorie in res.categories:
    categorie.id = bubble_client.create(
        "Category",
        {
            "Company": COMPANY_ID,
            "Description": categorie.description,
            "Name": categorie.name,
            #"Project": ,
        })

ValueError: "InsightCategory" object has no field "id"

In [ ]:
res = bubble_client.create(
        "python_insight",
        [{
            "company": company_id,
            "project": project_id,
            "content": row["content"],
            "backend_status": "new",
            "feedback_count": row["feedback_count"],
            "step": 1,
            "related_feedback":row['related_feedback'],
            "tag": row["tag"],
            "type": row["type"],
            "backend_type": row['backend_type'],
            "parents": row["parents"],
            "parent": str(row["parent"]),
            "childrens": 0,#[[] for _ in insight_layers[0][:1000].iterrows()],
            "cluster": row["cluster"],
        }  for _, row in insight_layers[0].iterrows()]
    )